<a href="https://colab.research.google.com/github/vishnubharadwaj1234/Test-Prep-Study-Tool/blob/main/TestPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install openai==0.28

In [ ]:
import openai
import gradio as gr
import gradio.components as components
import os

In [ ]:
openai.api_key = "ENTER OPENAI API KEY HERE"

messages = [
    {"role": "system", "content": "You are a friendly and helpful AI assistant."},
]

In [ ]:
previous_response = ""

def print_questions(subject, format):
    if subject:
        question = "Create 10 " + format + " questions for a test on " + subject + ". Do not write anything except for the questions."
        messages.append({"role": "user", "content": question})
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
        questions_reply = chat.choices[0].message.content
        messages.append({"role": "assistant", "content": questions_reply})
        previous_response = questions_reply
        return questions_reply

In [ ]:
def print_answers():
    prompt = "Give me the answers to the following questions: " + previous_response + "\n\nDo not write anything except for the answers."
    messages.append({"role": "user", "content": prompt})
    chat = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages
    )
    answers_reply = chat.choices[0].message.content
    messages.append({"role": "assistant", "content": answers_reply})
    return answers_reply

In [ ]:
def set_btn_interactive(btn):
  return gr.Button(interactive=True)

def set_textbox_interactive(txt):
  return gr.Textbox(visible=True)

In [ ]:
components_to_clear = []
questions_tb = ""

with gr.Blocks() as interface:
  with gr.Row():
    with gr.Column():
      subject = gr.Textbox(placeholder="Subject of test")
      format = gr.Dropdown(
          choices=["Multiple choice", "True/False", "Fill in the blank", "Short answer", "Free response"],
          allow_custom_value=True,
          label="Format of test",
          show_label=True)
      inputs = [subject, format]
      questions_btn = gr.Button(value="Create Questions", interactive=False)
      subject.input(fn=set_btn_interactive, inputs=questions_btn, outputs=questions_btn)
      questions_tb = gr.Textbox(lines=20, label="Your AI Generated Questions:")
      components_to_clear = [subject, format, questions_tb]
      questions_btn.click(fn=print_questions,
                inputs=inputs,
                outputs=questions_tb)


    with gr.Column():
      answer_area = gr.Textbox(label="Enter your answers here:", lines=12, visible=False)
      components_to_clear.append(answer_area)
      questions_tb.change(fn=set_textbox_interactive, inputs=answer_area, outputs=answer_area)

      answer_btn = gr.Button("Get Answers", interactive=False)
      answer_area.input(fn=set_btn_interactive, inputs=answer_btn, outputs=answer_btn)

      answers_tb = gr.Textbox(label="Answers to the questions:", lines=12)
      components_to_clear.append(answers_tb)
      answer_btn.click(fn=print_answers, inputs=[], outputs=answers_tb)

      clear_btn = gr.ClearButton()
      clear_btn.add(components_to_clear)

In [ ]:
interface.launch(share=True)